In [1]:
import tensorflow as tf
from transformers import AutoTokenizer
import pandas as pd
import re
import evaluate
from sklearn.model_selection import train_test_split
from transformers import T5ForConditionalGeneration
from tqdm import tqdm
from datasets import load_dataset
from transformers import TrainingArguments
from transformers import Trainer
from transformers import DataCollatorForSeq2Seq
import numpy as np
from transformers import TFT5ForConditionalGeneration

c:\Users\danendra\anaconda3\envs\tesis\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Load Model and Tokenizer

In [8]:
# Set up the T5 tokenizer
tokenizer = AutoTokenizer.from_pretrained("Wikidepia/IndoT5-base")

# Load the pre-trained T5 model yang sudah di fine tune sebelumnya
#jadi lanjutin training ini ceritanya
save_path = 'test-trainer'
model = T5ForConditionalGeneration.from_pretrained(save_path)

#ini kalau mau train dari pre-trained awal
#model = T5ForConditionalGeneration.from_pretrained("Wikidepia/IndoT5-base")

# Data

In [27]:
max_length = 128

def preprocess_function(examples):
    inputs = examples['clean_tweet']
    targets =examples["quadruplet"] 
    model_inputs = tokenizer(
        inputs, text_target=targets, max_length=max_length, truncation=True
    )
    return model_inputs

In [28]:
raw_dataset = load_dataset('csv', data_files='data/quadruplet_only.csv', split='train')
splitted_dataset = raw_dataset.train_test_split(test_size=0.1, seed=42)
preprocessed_dataset = splitted_dataset.map(
    preprocess_function,
    batched=True,
    remove_columns=raw_dataset.column_names,
    load_from_cache_file=False
)

Extracting data files: 100%|██████████| 1/1 [00:00<00:00, 133.23it/s]


Dataset csv downloaded and prepared to C:/Users/danendra/.cache/huggingface/datasets/csv/default-15ee197e989a3da1/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


In [41]:
raw_dataset

Dataset({
    features: ['original_id', 'original_tweet', 'clean_tweet', 'nolimit_sentiment', 'nolimit_aspect_categories', 'quadruplet', 'annotated_spam', 'model_spam'],
    num_rows: 39
})

In [9]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

# Train Model

In [10]:
training_args = TrainingArguments(
    save_path,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    resume_from_checkpoint=True,
    num_train_epochs=100,
    save_total_limit=2,
)

In [11]:
trainer = Trainer(
    model,
    training_args,
    train_dataset=preprocessed_dataset["train"],
    eval_dataset=preprocessed_dataset["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

In [12]:
trainer.train()

c:\Users\danendra\anaconda3\envs\tesis\lib\site-packages\transformers\optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
  0%|          | 0/4400 [00:00<?, ?it/s]You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
                                                  
  1%|          | 44/4400 [00:10<12:36,  5.76it/s]

{'eval_loss': 0.7464883923530579, 'eval_runtime': 0.2058, 'eval_samples_per_second': 189.459, 'eval_steps_per_second': 24.29, 'epoch': 1.0}


                                                   
  2%|▏         | 88/4400 [00:42<14:29,  4.96it/s]

{'eval_loss': 0.7376183867454529, 'eval_runtime': 0.2084, 'eval_samples_per_second': 187.178, 'eval_steps_per_second': 23.997, 'epoch': 2.0}


                                                   
  3%|▎         | 132/4400 [01:15<12:33,  5.67it/s]

...

                                                   
100%|██████████| 4400/4400 [59:30<00:00,  4.99it/s]

{'eval_loss': 1.1726911067962646, 'eval_runtime': 0.2418, 'eval_samples_per_second': 161.264, 'eval_steps_per_second': 20.675, 'epoch': 100.0}


100%|██████████| 4400/4400 [59:57<00:00,  1.22it/s]

{'train_runtime': 3597.0374, 'train_samples_per_second': 9.619, 'train_steps_per_second': 1.223, 'train_loss': 0.05623467380350286, 'epoch': 100.0}


TrainOutput(global_step=4400, training_loss=0.05623467380350286, metrics={'train_runtime': 3597.0374, 'train_samples_per_second': 9.619, 'train_steps_per_second': 1.223, 'train_loss': 0.05623467380350286, 'epoch': 100.0})

In [13]:
model.save_pretrained(save_path)

# Inference

## Hasil pytorch

In [42]:
tokenizer = AutoTokenizer.from_pretrained("Wikidepia/IndoT5-base")
model = T5ForConditionalGeneration.from_pretrained('models/pt-indot5').to('cuda')

In [46]:
test_df = pd.read_csv('data/quadruplet_test_pred.csv')
test_df.shape

(39, 9)

In [44]:
tokenized_text = tokenizer(test_df['clean_tweet'].values.tolist()
          , padding=True, truncation=True, return_tensors="pt")
generated_text = model.generate(tokenized_text['input_ids'].to('cuda'), max_length=max_length)
pred_text = tokenizer.batch_decode(generated_text, skip_special_tokens=True)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [47]:
test_df['pred_quadruplet_pt_t5'] = pred_text

In [49]:
test_df.to_csv('data/quadruplet_test_pred.csv', index=False)

## Hasil Tensorflow

In [50]:
tokenizer = AutoTokenizer.from_pretrained("Wikidepia/IndoT5-base")
model = TFT5ForConditionalGeneration.from_pretrained('models/tf-indot5')

c:\Users\danendra\anaconda3\envs\tesis\lib\site-packages\keras\initializers\initializers_v2.py:120: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(
All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at models/tf-indot5.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


In [51]:
test_df = pd.read_csv('data/quadruplet_test_pred.csv')
test_df.head()

,original_id,content,clean_tweet,final_sentiment,labels,quadruplet,spam,sentiment_label,pred_quadruplet_pt_bart,pred_quadruplet_pt_t5
0,1.640000e+18,@tokopedia min aku udah bayar tapi kenapa diba...,min aku udah bayar tapi kenapa dibatalin pesan...,negative,payment; produk;,"(pesananku, udah bayar tapi kenapa dibatalin, ...",NaN,sentiment,"(_, kenapa dibatalin pesanan ku, neutral, pay...","(pesananku, udah bayar tapi kenapa dibatalin, ..."
1,1.640000e+18,Seperti biasa menghibungi call center @sicepat...,Seperti biasa menghibungi call center hanya be...,negative,delivery; produk;,"(call center, hanya berbelit belit, negative, ...",NaN,sentiment,"(cs center, berbelit belit, negative, custome...","(call center, hanya berbelit belit, negative, ..."
2,1.640000e+18,@tanyakanrl alfa kalo jumat-minggu ada promo t...,alfa kalo jumat-minggu ada promo tuh jd murah ...,neutral,price; produk;,"(alfa, kalo jumat-minggu ada promo, positive, ...",NaN,sentiment,"( jumat-minggu, ada promo, positive, price); ...","(alfa, kalo jumat-minggu ada promo, positive, ..."
3,1.640000e+18,@tokopedia apakah tokopedia care sedang gangguan?,apakah tokopedia care sedang gangguan?,negative,website&apps; produk;,"(tokopedia, gangguan, negative, website&apps);",NaN,sentiment,"(tokopedia care, sedang gangguan, neutral, we...","(tokopedia care, sedang gangguan, neutral, web..."
4,1.640000e+18,"Beli token listrik lewat tokped error, direjec...","Beli token listrik lewat tokped error, direjec...",negative,website&apps; payment; produk;,"(token listrik, lewat tokped error, negative, ...",NaN,sentiment,"(esim listrik, belum solved beli esim xl, gab...","(token listrik, lewat tokped error, negative, ..."


In [52]:
tokenized_text = tokenizer(test_df['clean_tweet'].values.tolist()
          , padding=True, truncation=True, return_tensors="pt")
generated_text = model.generate(tokenized_text['input_ids'], max_length=max_length)
pred_text = tokenizer.batch_decode(generated_text, skip_special_tokens=True)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [55]:
test_df['pred_quadruplet_tf_t5'] = pred_text
test_df.to_csv('data/quadruplet_test_pred.csv', index=False)

# Post-process

In [28]:
from Levenshtein import distance

extract quadruplet

In [63]:
def extract_quadruplet(label):
    sample = [quadruplet.replace('(', '[').replace(')', ']') for quadruplet in label.split(';')]
    sample = [re.findall('\[(.*?)\]', quadruplet) for quadruplet in sample]
    quadruplets = []
    for quadruplet in sample:
        #untuk menghindari quadruplet kosong dari label hasil prediksi
        if quadruplet:
            quadruplet = quadruplet[0].split(',')
            quadruplets.append(tuple(quadruplet))
    return quadruplets

In [65]:
def compute_f1_scores(pred_pt, gold_pt):
    """
    Function to compute F1 scores with pred and gold pairs/triplets
    The input needs to be already processed
    """
    # number of true postive, gold standard, predicted aspect terms
    n_tp, n_gold, n_pred = 0, 0, 0

    for i in range(len(pred_pt)):
        n_gold += len(gold_pt[i])
        n_pred += len(pred_pt[i])

        for t in pred_pt[i]:
            if t in gold_pt[i]:
                n_tp += 1

    precision = float(n_tp) / float(n_pred) if n_pred != 0 else 0
    recall = float(n_tp) / float(n_gold) if n_gold != 0 else 0
    f1 = 2 * precision * recall / (precision + recall) if precision != 0 or recall != 0 else 0
    scores = {'precision': precision, 'recall': recall, 'f1': f1}

    return scores

In [64]:
idx = random.randint(0, len(x_test))
print(extract_quadruplet(pred_quadruplet[idx]))
print(extract_quadruplet(y_test[idx]))

[('service', ' quick', ' POS', ' service general')]
[('service', ' quick', ' POS', ' service general'), ('service', ' friendly', ' POS', ' service general')]


In [66]:
idx = random.randint(0, len(x_test))
pred_label = extract_quadruplet(pred_quadruplet[idx])
true_label = extract_quadruplet(y_test[idx])
compute_f1_scores(pred_label, true_label)

{'precision': 0.25, 'recall': 0.25, 'f1': 0.25}

In [29]:
distance("lewenstein", "levenshtein")

2

In [62]:
arr = ['a', 'b', 'c']
tuple(arr)

('a', 'b', 'c')